In [1]:
import numpy as np

### validação cruzada - conceito

Quando os dados são simplesmente separados em treino e teste em uma proporção fixa, além de a proporção de teste não ser utilizada no modelo, pode acontecer de haver overfitting com os dados de treino. A **validação cruzada** serve para aproveitar melhor os dados de treino e teste a fim de se evitar o overfitting.

**K-Fold**:O modo mais simples de fazer a validação cruzada é dividindo o conjunto de dados em k partes. Em cada iteração, escolhemos uma dessas partes para ser o conjunto de teste. As demais k-1 partes são utilizadas para treinar o modelo. O processo é repetido até que todas as k partes tenham sido utilizadas como teste.

O **Stratified K-Fold** é usado quando o conjunto de dados é desbalanceado. Ele també funciona dividindo em fold's, mas sempre mantendo o mesmo percentual do envento (ou não evento) raro na base.

O **Leave-One-Out** funciona utilizando apenas um exemplo como teste e o restante do conjunto de dados como treino. Na prática é pouco utilizado.

In [2]:
X = np.asarray([2, 6, 8, 8, 12, 16, 20, 20, 22, 26]).reshape(-1, 1)
y = np.asarray([58, 105, 88, 118, 117, 137, 157, 169, 149, 202])

In [3]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5)

for i in kfold.split(X):
    print(f'treino: {i[0]}\n teste: {i[1]}')

treino: [2 3 4 5 6 7 8 9]
 teste: [0 1]
treino: [0 1 4 5 6 7 8 9]
 teste: [2 3]
treino: [0 1 2 3 6 7 8 9]
 teste: [4 5]
treino: [0 1 2 3 4 5 8 9]
 teste: [6 7]
treino: [0 1 2 3 4 5 6 7]
 teste: [8 9]


In [4]:
kfold = KFold(n_splits = 5)

for train_index, test_index in kfold.split(X):
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print(f'\nModelo: y = {lr.coef_[0]} x + {lr.intercept_}')


Modelo: y = 5.149999999999999 x + 59.40000000000002


In [5]:
from sklearn.model_selection import cross_validate

lr = LinearRegression()

# quando um inteiro é passado no hiperparâmetro cv o algoritmo faz um stratified kfold
cross_validate(lr, X, y, scoring = 'r2', cv = 3)

{'fit_time': array([0.00299692, 0.0009985 , 0.00099897]),
 'score_time': array([0.        , 0.00100112, 0.        ]),
 'test_score': array([0.40081415, 0.92639764, 0.52379637])}

In [6]:
cross_validate(lr, X, y, scoring = 'r2', cv = kfold)

{'fit_time': array([0.0009985 , 0.00099897, 0.0009985 , 0.00099921, 0.00100136]),
 'score_time': array([0.0009973 , 0.        , 0.        , 0.00099945, 0.        ]),
 'test_score': array([ 0.66211164, -0.08825126,  0.859375  , -0.55157035,  0.54618726])}

In [7]:
# é possível retornar todos os estimadores treinados, assim como mais de uma métrica

cross_validate(lr, X, y, scoring = ['r2', 'neg_mean_squared_error'], cv = kfold, return_estimator = True)

{'fit_time': array([0.00101757, 0.00099874, 0.00099206, 0.00099945, 0.00100088]),
 'score_time': array([0.00097919, 0.00200009, 0.00099969, 0.00199819, 0.00100017]),
 'estimator': [LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression(),
  LinearRegression()],
 'test_r2': array([ 0.66211164, -0.08825126,  0.859375  , -0.55157035,  0.54618726]),
 'test_neg_mean_squared_error': array([-186.59884495, -244.85653262,  -14.0625    ,  -55.85653262,
        -318.69      ])}